In [8]:
save_path = "./test_images"
metadata_save_name = save_path + "/metadata.csv"
img_save_path = save_path + "/structure_examples"

In [9]:
import quilt3 as q3
q3.Package.browse(
    "aics/hipsc_single_cell_image_dataset",
    registry="s3://allencell"
)["metadata.csv"].fetch(metadata_save_name)

Loading manifest: 100%|██████████| 484465/484465 [00:07<00:00, 63.0k/s]
100%|██████████| 1.69G/1.69G [01:34<00:00, 17.8MB/s] 


PackageEntry('file:///D:/Projects/004_Cellpose/Allen_Cell_and_Structure_Segmenter/test_images/metadata.csv')

In [11]:
# 对于每种 structure_name：
# 下载 fov_path 对应的图像，提取结构通道 ChannelNumberStruct，保存为 结构名_raw.tif
# 下载 struct_seg_path 对应的分割图像，保存为 结构名_seg.tif
# 文件统一保存在 ./test_images/structure_examples/ 文件夹中
# 如果遇到问题，访问下一条记录
# 如果代码运行一直出错可能是网络问题
import pandas as pd
import quilt3 as q3
from pathlib import Path
from aicsimageio import AICSImage
from tifffile import imwrite

# 参数
MAX_TRIES = 10
output_dir = Path(img_save_path)
output_dir.mkdir(parents=True, exist_ok=True)

# 加载 metadata
df = pd.read_csv(metadata_save_name)
df_unique = df.drop_duplicates(subset="fov_path")

# quilt 包
pkg = q3.Package.browse("aics/hipsc_single_cell_image_dataset", registry="s3://allencell")

# 遍历每个结构
for structure, group in df_unique.groupby("structure_name"):
    print(f"\n▶ 正在处理结构: {structure}")
    success = False

    for _, row in group.head(MAX_TRIES).iterrows():
        try:
            # 下载并读取 fov 图像
            fov_path = row["fov_path"]
            local_fov = output_dir / "temp_fov.ome.tiff"
            if not local_fov.exists():
                pkg[fov_path].fetch(local_fov)
            img = AICSImage(local_fov)
            ch_idx = int(row["ChannelNumberStruct"])
            raw = img.get_image_data("ZYX", C=ch_idx)

            # 下载并读取 seg 图像
            seg_path = row["struct_seg_path"]
            local_seg = output_dir / "temp_seg.ome.tiff"
            if not local_seg.exists():
                pkg[seg_path].fetch(local_seg)
            seg = AICSImage(local_seg).get_image_data("ZYX")

            # 保存输出
            raw_out = output_dir / f"{structure}_raw.tif"
            seg_out = output_dir / f"{structure}_seg.tif"
            imwrite(raw_out, raw)
            imwrite(seg_out, seg)
            print(f"✅ 成功保存: {raw_out} 和 {seg_out}")

            # 删除临时文件
            local_fov.unlink(missing_ok=True)
            local_seg.unlink(missing_ok=True)

            success = True
            break
        except Exception as e:
            print(f"⚠️ 失败: {e}")
            continue

    if not success:
        print(f"❌ {structure} 的前 {MAX_TRIES} 条记录全部失败")


C:\Users\Xiaofei\AppData\Local\Temp\ipykernel_45860\3351000164.py:19: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_save_name)
Loading manifest: 100%|██████████| 484465/484465 [00:07<00:00, 65.0k/s]



▶ 正在处理结构: AAVS1


100%|██████████| 423M/423M [00:48<00:00, 8.76MB/s]  
100%|██████████| 615k/615k [00:09<00:00, 63.9kB/s]
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ 成功保存: test_images\structure_examples\AAVS1_raw.tif 和 test_images\structure_examples\AAVS1_seg.tif

▶ 正在处理结构: ACTB


100%|██████████| 441M/441M [00:44<00:00, 9.85MB/s]  
100%|██████████| 469k/469k [00:09<00:00, 49.9kB/s]
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


...

✅ 成功保存: test_images\structure_examples\TJP1_raw.tif 和 test_images\structure_examples\TJP1_seg.tif

▶ 正在处理结构: TOMM20


100%|██████████| 372M/372M [00:21<00:00, 17.4MB/s]  
100%|██████████| 599k/599k [00:04<00:00, 127kB/s] 
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ 成功保存: test_images\structure_examples\TOMM20_raw.tif 和 test_images\structure_examples\TOMM20_seg.tif

▶ 正在处理结构: TUBA1B


100%|██████████| 300M/300M [00:19<00:00, 15.6MB/s]  
100%|██████████| 1.21M/1.21M [00:04<00:00, 301kB/s]
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid PixelsID '0' to 'Pixels:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
c:\Users\Xiaofei\.conda\envs\quilt-env\lib\site-packages\pydantic\main.py:253: UserWarning: Casting invalid ImageID '0' to 'Image:0'
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


✅ 成功保存: test_images\structure_examples\TUBA1B_raw.tif 和 test_images\structure_examples\TUBA1B_seg.tif
